# README – Dataset Sintético de Trigo (La Araucanía)

## 1. Descripción
Este dataset sintético fue generado para proyectos de predicción del **rendimiento de trigo** usando modelos de **regresión supervisada** (MLP).  
Incluye variables climáticas, de suelo, manejo agrícola y presencia de plagas/deficiencias, y un **rendimiento estimado en kg/ha** como variable objetivo.

El objetivo es proporcionar un **dataset coherente y realista**, con interacciones agronómicas simuladas, para entrenar modelos de machine learning sin necesidad de datos reales.

---

## 2. Número de registros
- **Filas:** 2.000  
- **Columnas:** 14  

---

## 3. Variables

| Nombre | Tipo | Descripción | Rango / Valores |
|--------|------|------------|----------------|
| Temporada | Categórica | Estación de cultivo | `"invierno"` / `"primavera"` |
| Lluvia_mm | Numérica | Lluvia acumulada durante la temporada | Invierno ~ 654 mm ±50, Primavera ~ 253 mm ±30 |
| Horas_luz | Numérica | Horas de luz promedio | Invierno ~ 84 h ±5, Primavera ~ 69 h ±5 |
| Temp_media | Numérica | Temperatura media (°C) | Invierno ~ 15 ±2, Primavera ~ 19 ±2 |
| Temp_max | Numérica | Temperatura máxima (°C) | Media + 3–6 °C |
| Temp_min | Numérica | Temperatura mínima (°C) | Media - 3–6 °C |
| N_kg_ha | Numérica | Nitrógeno aplicado (kg/ha) | 120 – 256 |
| P_kg_ha | Numérica | Fósforo aplicado (kg/ha) | 70 – 100 |
| K_kg_ha | Numérica | Potasio aplicado (kg/ha) | 80 – 100 |
| pH | Numérica | Acidez del suelo | 6.0 – 8.5 (normal centrado en 6.8) |
| Labranza | Categórica | Tipo de labranza | `"convencional"` / `"cero labranza"` |
| Plagas | Categórica | Presencia de plagas | `"sí"` / `"no"` |
| Deficiencia | Categórica | Deficiencia de nutrientes (azufre/boro) | `"sí"` / `"no"` |
| Rendimiento_kg_ha | Numérica | Rendimiento estimado del cultivo | ~1000 – 12000 kg/ha |

---

## 4. Notas sobre la generación de datos
1. **Efectos climáticos**: lluvia, horas de luz y temperatura afectan el rendimiento de manera pronunciada.  
2. **Nutrientes**: N, P y K tienen efecto **logarítmico**, simulando rendimientos marginales decrecientes.  
3. **pH**: efecto tipo campana centrado en 6.8.  
4. **Fertilizante fosforado**: efecto dependiente del N disponible y de la cantidad de lluvia.  
5. **Labranza**: penalización mayor en cero labranza si N es bajo o en primavera.  
6. **Plagas y deficiencias**: reducen el rendimiento y dependen parcialmente de la temporada.  
7. **Ruido aleatorio**: simula variabilidad natural (~10% del rendimiento esperado).  


In [25]:
# !pip install numpy

In [26]:
# !pip install pandas

In [38]:
import numpy as np
import pandas as pd

In [39]:
# Función del efecto del fertilizante fosforado según N y lluvia
def efecto_fosforado(N, lluvia, base=50, max_N=256, lluvia_opt=400):
    """
    Fertilizante fosforado potencia el rendimiento si N es bajo y hay suficiente lluvia.
    """
    factor_N = max(0, 1 - N / max_N)
    factor_lluvia = min(1, lluvia / lluvia_opt)  # reduce efecto si hay poca lluvia
    return base * factor_N * factor_lluvia

# Función de rendimiento sintético mejorada
def calcular_rendimiento(
    temporada, lluvia, horas_luz, temp_media, temp_max, temp_min,
    N, P, K, pH, labranza, plagas, deficiencia
):
    # # --- Clima ---
    # efecto_lluvia = 0.8 * lluvia
    # efecto_luz = 20 * horas_luz / 100
    # efecto_temp = -10 * ((temp_media - 18)/10)**2

    # --- Clima con efecto más fuerte ---
    efecto_lluvia = 1.5 * lluvia            # antes era 0.8
    efecto_luz = 50 * horas_luz / 100      # antes era 20
    efecto_temp = -30 * ((temp_media - 18)/10)**2  # antes era -10

    
    # --- Nutrientes logarítmicos ---
    efecto_N = 200 * np.log(N / 50)
    efecto_P = 50 * np.log(P / 70)
    efecto_K = 30 * np.log(K / 80)
    
    # --- pH tipo campana ---
    efecto_pH = -100 * (pH - 6.8)**2 + 150
    
    # --- Labranza con interacción ---
    if labranza == "convencional":
        efecto_labranza = 0
    else:
        # cero labranza penaliza más si N es bajo y en primavera
        efecto_labranza = -50 - 0.2*(256 - N)
        if temporada == "primavera":
            efecto_labranza -= 50
    
    # --- Fertilizante fosforado ---
    efecto_fertilizante = efecto_fosforado(N, lluvia)
    
    # --- Plagas y deficiencia (dependientes de temporada) ---
    efecto_plagas = -200 if plagas == "sí" else 0
    efecto_deficiencia = -100 if deficiencia == "sí" else 0
    
    # --- Rendimiento base ---
    rendimiento_base = (
        3000 + efecto_lluvia + efecto_luz + efecto_temp +
        efecto_N + efecto_P + efecto_K + efecto_pH +
        efecto_labranza + efecto_fertilizante +
        efecto_plagas + efecto_deficiencia
    )
    
    # Ruido proporcional (10% del rendimiento base)
    ruido = np.random.normal(0, 0.1 * rendimiento_base)
    
    rendimiento_total = max(rendimiento_base + ruido, 1000)
    
    return rendimiento_total

In [40]:
# --- Generar dataset sintético ---
n_datos = 50000
data = []

for _ in range(n_datos):
    temporada = np.random.choice(["invierno", "primavera"])
    
    # Clima
    if temporada == "invierno":
        lluvia = np.random.normal(654.9, 50)
        horas_luz = np.random.normal(84.2, 5)
        temp_media = np.random.normal(15.4, 2)
        temp_max = temp_media + np.random.uniform(3, 6)
        temp_min = temp_media - np.random.uniform(3, 6)
    else:
        lluvia = np.random.normal(253, 30)
        horas_luz = np.random.normal(68.7, 5)
        temp_media = np.random.normal(19.2, 2)
        temp_max = temp_media + np.random.uniform(3, 6)
        temp_min = temp_media - np.random.uniform(3, 6)
    
    # Nutrientes
    N = np.random.uniform(120, 256)
    P = np.random.uniform(70, 100)
    K = np.random.uniform(80, 100)
    
    # pH centrado alrededor de 6.8
    pH = np.random.normal(6.8, 0.5)
    pH = np.clip(pH, 5.8, 8.5)
    
    # Labranza
    labranza = np.random.choice(["convencional", "cero labranza"])
    
    # Plagas (más probables en primavera)
    plagas_prob = 0.3 if temporada=="primavera" else 0.1
    plagas = np.random.choice(["sí", "no"], p=[plagas_prob, 1-plagas_prob])
    
    # Deficiencia
    deficiencia = np.random.choice(["sí", "no"], p=[0.1, 0.9])
    
    # Calcular rendimiento
    rendimiento = calcular_rendimiento(
        temporada, lluvia, horas_luz, temp_media, temp_max, temp_min,
        N, P, K, pH, labranza, plagas, deficiencia
    )
    
    data.append([
        temporada, lluvia, horas_luz, temp_media, temp_max, temp_min,
        N, P, K, pH, labranza, plagas, deficiencia, rendimiento
    ])

# Crear DataFrame y guardar CSV
columnas = [
    "Temporada", "Lluvia_mm", "Horas_luz", "Temp_media", "Temp_max", "Temp_min",
    "N_kg_ha", "P_kg_ha", "K_kg_ha", "pH", "Labranza", "Plagas", "Deficiencia", "Rendimiento_kg_ha"
]
df = pd.DataFrame(data, columns=columnas)
df.to_csv("trigo_sintetico_mejorado.csv", index=False)
print("Dataset mejorado generado: trigo_sintetico_mejorado.csv")

Dataset mejorado generado: trigo_sintetico_mejorado.csv


In [1]:
import numpy as np
import pandas as pd

# -------------------------
# Fórmula compleja de rendimiento
# -------------------------
def calcular_rendimiento_complex(
    temporada, lluvia, horas_luz, temp_media, temp_max, temp_min,
    N, P, K, pH, labranza, plagas, deficiencia,
    base=1200,
    K_lluvia=300.0, K_N=100.0, K_P=50.0, K_K=40.0,
    T_opt_invierno=15.0, T_opt_primavera=19.0, sigma_temp=3.0,
    plaga_penalty_frac=0.20, def_penalty_frac=0.12,
    labranza_boost_conv=1.06, labranza_boost_cero=1.0,
    interact_temp_N=0.002, interact_lluvia_horas=0.0005,
    pH_opt=6.8, pH_penalty_coeff=60.0,
    max_scale=10000.0, noise_frac=0.06
):
    # Saturación lluvia y nutrientes
    lluvia_eff = lluvia / (K_lluvia + lluvia)
    N_eff = N / (K_N + N)
    P_eff = P / (K_P + P)
    K_eff = K / (K_K + K)
    nutrientes_eff = (N_eff + P_eff + K_eff)

    # Temperatura gaussiana
    T_opt = T_opt_invierno if temporada == "invierno" else T_opt_primavera
    temp_factor = np.exp(-0.5 * ((temp_media - T_opt) / sigma_temp) ** 2)

    # Horas luz saturadas
    horas_eff = horas_luz / (1.0 + 0.01 * horas_luz)

    # Penalización pH
    pH_penalty = - pH_penalty_coeff * (pH - pH_opt) ** 2

    # Componentes principales
    clima_component = 2500.0 * lluvia_eff + 8.0 * horas_eff
    nutrientes_component = 1200.0 * nutrientes_eff
    rendimiento_det = base + clima_component * temp_factor + nutrientes_component + pH_penalty

    # Interacciones
    rendimiento_det += interact_temp_N * temp_media * N
    rendimiento_det += interact_lluvia_horas * lluvia * horas_luz

    # Umbrales condicionales
    if temporada == "invierno" and lluvia > 800 and temp_media < (T_opt - 1.5):
        rendimiento_det -= 400.0
    if N > 220 and P > 90 and K > 95:
        rendimiento_det += 150.0

    # Labranza
    rendimiento_det *= labranza_boost_conv if labranza == "convencional" else labranza_boost_cero

    # Penalizaciones plagas/deficiencia
    if plagas == "sí":
        rendimiento_det *= (1.0 - plaga_penalty_frac)
    if deficiencia == "sí":
        rendimiento_det *= (1.0 - def_penalty_frac)

    # Límite valores
    rendimiento_det = max(rendimiento_det, 300.0)
    rendimiento_det = min(rendimiento_det, max_scale)

    # Ruido proporcional
    sigma_noise = noise_frac * rendimiento_det
    rendimiento_final = rendimiento_det + np.random.normal(0, sigma_noise)

    return float(max(rendimiento_final, 200.0))


# -------------------------
# Generación del dataset
# -------------------------
n_datos = 1000  # puedes cambiar a 50000, 10000, etc.
data = []

for _ in range(n_datos):
    temporada = np.random.choice(["invierno", "primavera"])

    # Clima según temporada
    if temporada == "invierno":
        lluvia = np.random.normal(654.9, 50)
        horas_luz = np.random.normal(84.2, 5)
        temp_media = np.random.normal(15.4, 2)
    else:
        lluvia = np.random.normal(253, 30)
        horas_luz = np.random.normal(68.7, 5)
        temp_media = np.random.normal(19.2, 2)

    temp_max = temp_media + np.random.uniform(3, 6)
    temp_min = temp_media - np.random.uniform(3, 6)

    # Nutrientes
    N = np.random.uniform(120, 256)
    P = np.random.uniform(70, 100)
    K = np.random.uniform(80, 100)

    # pH
    pH = np.clip(np.random.normal(6.8, 0.5), 5.8, 8.5)

    # Categóricas
    labranza = np.random.choice(["convencional", "cero labranza"])
    plagas_prob = 0.3 if temporada == "primavera" else 0.1
    plagas = np.random.choice(["sí", "no"], p=[plagas_prob, 1 - plagas_prob])
    deficiencia = np.random.choice(["sí", "no"], p=[0.1, 0.9])

    # Rendimiento con fórmula compleja
    rendimiento = calcular_rendimiento_complex(
        temporada, lluvia, horas_luz, temp_media, temp_max, temp_min,
        N, P, K, pH, labranza, plagas, deficiencia
    )

    # -------------------------
    # Features adicionales
    # -------------------------
    GDD = max(0, temp_media - 5) * (90 if temporada == "invierno" else 120)  # Growing Degree Days
    rango_termico = temp_max - temp_min
    fertilidad_total = N + P + K
    categoria_clima = "óptimo" if (lluvia > 400 and 14 <= temp_media <= 20) else "estresado"
    pH_categorizado = "ácido" if pH < 6.3 else ("alcalino" if pH > 7.3 else "óptimo")

    data.append([
        temporada, lluvia, horas_luz, temp_media, temp_max, temp_min,
        N, P, K, pH, labranza, plagas, deficiencia,
        GDD, rango_termico, fertilidad_total, categoria_clima, pH_categorizado,
        rendimiento
    ])

# Crear DataFrame y guardar
columnas = [
    "Temporada", "Lluvia_mm", "Horas_luz", "Temp_media", "Temp_max", "Temp_min",
    "N_kg_ha", "P_kg_ha", "K_kg_ha", "pH", "Labranza", "Plagas", "Deficiencia",
    "GDD", "Rango_termico", "Fertilidad_total", "Categoria_clima", "pH_categorizado",
    "Rendimiento_kg_ha"
]

df = pd.DataFrame(data, columns=columnas)
df.to_csv("Dataset_Demostracion.csv", index=False)
print("✅ Dataset generado: trigo_sintetico_complejo.csv")
print("Tamaño:", df.shape)


✅ Dataset generado: trigo_sintetico_complejo.csv
Tamaño: (1000, 19)
